In [1]:
import os
import requests
import re
import sys
from bs4 import BeautifulSoup


In [2]:
def get_page():
    global url

    url = input("Enter url of a medium article: ").strip()

    # Allow direct Medium OR archived Medium URLs
    if not re.match(r'https?://(web\.archive\.org/web/.*/)?https?://medium\.com/', url):
        print('Please enter a valid Medium article (direct or archived)')
        sys.exit(1)

    res = requests.get(url)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, 'html.parser')
    return soup


In [3]:
def clean(text):
    rep = {"<br>": "\n", "<br/>": "\n", "<li>": "\n"}
    rep = dict((re.escape(k), v) for k, v in rep.items())
    pattern = re.compile("|".join(rep.keys()))
    text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
    text = re.sub('\<(.*?)\>', '', text)
    return text


In [4]:
def collect_text(soup):
    text = f'url: {url}\n\n'
    para_text = soup.find_all('p')

    for para in para_text:
        text += f"{para.text}\n\n"

    return text


In [5]:
def save_file(text):
    if not os.path.exists('./scraped_articles'):
        os.mkdir('./scraped_articles')

    name = url.split("/")[-1]
    fname = f'scraped_articles/{name}.txt'

    with open(fname, 'w', encoding='utf-8') as f:
        f.write(text)

    print(f'File saved in directory {fname}')


In [6]:
text = collect_text(get_page())
save_file(text)


Enter url of a medium article:  https://web.archive.org/web/20191126074327/https://medium.com/@subashgandyer/papa-what-is-a-neural-network-c5e5cc427c7


File saved in directory scraped_articles/papa-what-is-a-neural-network-c5e5cc427c7.txt
